<img src=https://static.wixstatic.com/media/f6bd9a_6a972d9b31324653bf198c38ce94339e~mv2.png>

# Carrusel de RDBMS
## Ejercicios de SQL resueltos

En este cuaderno resuelvo los ejercicios del 51 al 60 del documento: 'Desarrollo de Aplicaciones en Entornos de Cuarta Generación y Herramientas CASE: Ejercicios de SQL.'

Para ello utilizo una version de la base de datos  modificada para ser compatible en esta ocasion, para **Oracle**, siguiendo la estructura del libro.


Tanto el diagrama de **Entidad-Relacion**, como el **Script** para cargar la base de datos (que permite, se pueda replicar el ejercicio) se encuentran disponibles en el mismo repositorio.

#### Celda para configurar el entorno

In [ ]:
# !python -m pip install oracledb
# !pip install pandas
# !pip install sqlalchemy

#### Importamos librerias y se realiza la conexion utilizando un archivo json con la configuracion local de la base de datos

Si se desea replicar lo propio usando el script y este cuaderno, basta con crear el archivo json con el par de valores que contengan su informacion local y cargarlo al directorio.

host:

user:

password:

sid:

In [18]:
# importar las librerias
import json
from sqlalchemy import create_engine
from sqlalchemy import text
import pandas as pd

# Cargar los detalles de la conexión desde un archivo de configuración propio
with open('config3.json') as f:
    config = json.load(f)

# Crear la cadena de conexión
connection_str = f"oracle+oracledb://{config['username']}:{config['password']}@{config['host']}:{config['port']}/{config['sid']}"

# Crear objeto engine de sqlalchemy que gestiona la conexion
engine = create_engine(connection_str)

#### Realizamos una prueba y se Inspecciona la base de datos

In [2]:
# Definir consulta en SQL compatible con Oracle
try:
    QUERY = """
    SELECT table_name FROM user_tables
    """
# Ejecutar la consulta y obtener los resultados en un DataFrame
    df = pd.read_sql_query(QUERY, engine)
# Mostrar los primeros resultados
    print("La conexión se ha realizado con éxito.\n")
    print(df)
except Exception as e:
    print("Hubo un error al realizar la conexión:", e)

La conexión se ha realizado con éxito.

     table_name
0      HOSPITAL
1          SALA
2     PLANTILLA
3     OCUPACION
4        DOCTOR
5       ENFERMO
6      EMPLEADO
7  DEPARTAMENTO


#### Inspeccion a las tablas de la base de datos 
#### Con la finalidad de observar su estructura y tipo de datos

In [3]:
for valor in df['table_name']:
    QUERY = f"SELECT column_name, data_type, data_length FROM USER_TAB_COLUMNS WHERE table_name = '{valor}'" 
    df1 = pd.read_sql_query(QUERY, engine)
    print(f'El contenido de la tabla {valor}\n')
    print(df1)
    print()

El contenido de la tabla HOSPITAL

    column_name data_type  data_length
0  HOSPITAL_COD    NUMBER           22
1        NOMBRE  VARCHAR2           10
2     DIRECCION  VARCHAR2           20
3      TELEFONO  VARCHAR2            8
4      NUM_CAMA    NUMBER           22

El contenido de la tabla SALA

    column_name data_type  data_length
0  HOSPITAL_COD    NUMBER           22
1      SALA_COD    NUMBER           22
2        NOMBRE  VARCHAR2           20
3      NUM_CAMA    NUMBER           22

El contenido de la tabla PLANTILLA

    column_name data_type  data_length
0  HOSPITAL_COD    NUMBER           22
1      SALA_COD    NUMBER           22
2   EMPLEADO_NO    NUMBER           22
3      APELLIDO  VARCHAR2           15
4       FUNCION  VARCHAR2           10
5         TURNO  VARCHAR2            1
6       SALARIO    NUMBER           22

El contenido de la tabla OCUPACION

    column_name data_type  data_length
0   INSCRIPCION    NUMBER           22
1  HOSPITAL_COD    NUMBER           22
2

### Resolucion de los ejercicios numerados del 51 al 60

##### 51. Se desea averiguar el numero de oficios por departamento, sin tener en cuenta en el muestreo a aquellos individuos que están en alguno de los departamentos que contienen VENDEDORES. La salida de la consulta será como la siguiente.

|NUMERO|NUM-DEP|NOMBRE|
|:---:|:--------:|:---------:|
|3 |10 |CONTABILIDAD|
|3 |20 |INVESTIGACIÓN|
|0 |40 |OPERACIONES|

In [4]:
QUERY = """
SELECT  COUNT(DISTINCT(E.OFICIO)) AS NUMERO, D.DEPT_NO AS NOM_DEP, D.DNOMBRE AS NOMBRE
FROM EMPLEADO E
RIGHT OUTER JOIN DEPARTAMENTO D
ON E.DEPT_NO = D.DEPT_NO
WHERE D.DEPT_NO NOT IN(SELECT DISTINCT(DEPT_NO)
                        FROM EMPLEADO
                        WHERE UPPER(OFICIO) LIKE 'VENDEDO%')
GROUP BY  D.DEPT_NO, D.DNOMBRE
ORDER BY 2
"""
df = pd.read_sql_query(QUERY, engine)
df.columns = df.columns.str.upper()
df

,NUMERO,NOM_DEP,NOMBRE
0,3,10,CONTABILIDAD
1,3,20,INVESTIGACIÓN
2,0,40,OPERACIONES


##### 52. Sacar con el formato que aparece abajo, el apellido departamento y sueldo del empleado que mas gana en la empresa y del que menos.

|DEP|MAXIMO|APELLIDO|DEP|MÍNIMO|APELLIDO|
|:------:|:------------:|:---------:|:-------:|:------------:|:-------:|
|10|650000|REY|20|104000|SANCHEZ|

In [5]:
QUERY = """
SELECT MX.DEPT_NO DEP, MX.SALARIO MAXIMO, MX.APELLIDO APELLIDO, MN.DEPT_NO DEP,  MN.SALARIO MINIMO, MN.APELLIDO APELLIDO
FROM
(SELECT DEPT_NO, APELLIDO, SALARIO FROM EMPLEADO WHERE SALARIO = (SELECT MAX(SALARIO) FROM EMPLEADO)) MX,
(SELECT DEPT_NO, APELLIDO, SALARIO FROM EMPLEADO WHERE SALARIO = (SELECT MIN(SALARIO) FROM EMPLEADO)) MN
"""
df = pd.read_sql_query(QUERY, engine)
df.columns = df.columns.str.upper()
df

,DEP,MAXIMO,APELLIDO,DEP,MINIMO,APELLIDO
0,10,650000,Rey,20,104000,Sanchez


##### 53. En que departamento se dio de alta a mas empleados en Diciembre.

In [6]:
QUERY = """
SELECT * 
FROM (SELECT D.DNOMBRE NOMBRE, COUNT(E.DEPT_NO) TOTAL
        FROM DEPARTAMENTO D
        INNER JOIN EMPLEADO E
        ON D.DEPT_NO = E.DEPT_NO
        WHERE EXTRACT(MONTH FROM FECHA_ALTA) = 12
        GROUP BY D.DNOMBRE
        ORDER BY TOTAL DESC)
WHERE ROWNUM <= 1
"""
df = pd.read_sql_query(QUERY, engine)
df.columns = df.columns.str.upper()
df

,NOMBRE,TOTAL
0,INVESTIGACIÓN,3


##### 54. Se desea obtener, para cada departamento, su mínimo y su maximo salarial. Para ello, no se tendrá en cuenta a los empleados cuya primera letra de su apellido, coincida con la inicial del nombre del departamento en que trabajan. Asimismo, se tendrá en cuenta a aquellos departamentos cuya diferencia entre el maximo y el mínimo exceda la media salarial de toda la empresa

In [7]:
QUERY = """
SELECT DEPT_NO, MAX(SALARIO) MAXIMO, MIN(SALARIO) MINIMO
FROM EMPLEADO
WHERE EMP_NO IN (SELECT E.EMP_NO
                FROM EMPLEADO E
                INNER JOIN DEPARTAMENTO D
                ON E.DEPT_NO = D.DEPT_NO
                WHERE SUBSTR(E.APELLIDO,1,1) <> SUBSTR(D.DNOMBRE,1,1))
GROUP BY DEPT_NO
HAVING MAX(SALARIO) - MIN(SALARIO)  > (SELECT AVG(SALARIO) FROM EMPLEADO)
"""
df = pd.read_sql_query(QUERY, engine)
df.columns = df.columns.str.upper()
df

,DEPT_NO,MAXIMO,MINIMO
0,20,390000,104000
1,10,650000,169000


##### 55. Queremos saber el nombre de el empleado mas joven de cada departamento, así como el nombre de este.

In [8]:
QUERY = """
SELECT E.DEPT_NO, D.DNOMBRE, E.APELLIDO, E.FECHA_ALTA
FROM EMPLEADO E
INNER JOIN DEPARTAMENTO D ON E.DEPT_NO = D.DEPT_NO
INNER JOIN (SELECT  DEPT_NO, MAX(FECHA_ALTA) F FROM EMPLEADO GROUP BY DEPT_NO) C
ON E.DEPT_NO = C.DEPT_NO AND E.FECHA_ALTA = C.F
ORDER BY 1
"""
df = pd.read_sql_query(QUERY, engine)
df.columns = df.columns.str.upper()
df

,DEPT_NO,DNOMBRE,APELLIDO,FECHA_ALTA
0,10,CONTABILIDAD,Muñoz,1982-01-23
1,20,INVESTIGACIÓN,Alonso,1983-01-12
2,30,VENTAS,Jimeno,1981-12-03


##### 56. Se desea saber el nombre, oficio y departamento del empleado que más gana del departamento con la media salarial más alta.

In [9]:
QUERY = """
SELECT DEPT_NO, APELLIDO, OFICIO
FROM EMPLEADO
WHERE DEPT_NO =
                (SELECT DEPT_NO FROM
                                (SELECT DEPT_NO, ROUND(AVG(SALARIO), 0) FROM EMPLEADO GROUP BY DEPT_NO ORDER BY 2 DESC)
                WHERE ROWNUM <= 1)
AND SALARIO IN (SELECT MAX(SALARIO) FROM EMPLEADO GROUP BY DEPT_NO)
"""
df = pd.read_sql_query(QUERY, engine)
df.columns = df.columns.str.upper()
df

,DEPT_NO,APELLIDO,OFICIO
0,10,Rey,Presidente


##### 57. Se desea obtener información sobre todos los empleados que son jefes de alguien.

In [10]:
QUERY = """
SELECT APELLIDO, OFICIO, DEPT_NO
FROM EMPLEADO 
WHERE EMP_NO IN 
                (SELECT DISTINCT(DIR)
                FROM EMPLEADO 
                WHERE DIR IS NOT NULL)
ORDER BY 1
"""
df = pd.read_sql_query(QUERY, engine)
df.columns = df.columns.str.upper()
df

,APELLIDO,OFICIO,DEPT_NO
0,Cerezo,Director,10
1,Fernandez,Analista,20
2,Gil,Analista,20
3,Jimenez,Director,20
4,Negro,Director,30
5,Rey,Presidente,10


##### 58. Recuperar el numero (empleado_no) y nombre de las personas que perciban un salario > que la media de **su** hospital.

In [11]:
QUERY = """
SELECT P.EMPLEADO_NO, P.APELLIDO
FROM PLANTILLA P, 
                (SELECT HOSPITAL_COD, ROUND(AVG(SALARIO),0) AS SAVG
                FROM PLANTILLA
                GROUP BY HOSPITAL_COD) P2
WHERE P.HOSPITAL_COD = P2.HOSPITAL_COD AND P.SALARIO > P2.SAVG
"""
df = pd.read_sql_query(QUERY, engine)
df.columns = df.columns.str.upper()
df

,EMPLEADO_NO,APELLIDO
0,3106,Hernandez J.
1,1009,Higueras D.
2,9901,NuÑez C.
3,7379,Carlos R.
4,8526,Frank H.


#### INSERCIONES

##### 59. Insertar en la tabla Plantilla al Garcia J. con un sueldo de 3000000 ptas, y número de empleado 1234. Trabaja en el hospital 22, sala2.

In [12]:
with engine.begin() as connection:
    QUERY = """
    INSERT INTO PLANTILLA (HOSPITAL_COD,SALA_COD, EMPLEADO_NO, APELLIDO, FUNCION, TURNO, SALARIO)
    VALUES (22, 2, 1234, 'Garcia J.',NULL,NULL , 3000000)
    """
    result = connection.execute(text(QUERY))
print(f' se ha modificado {result.rowcount} fila(s)')

 se ha modificado 1 fila(s)


Como el siguiente ejercicio requiere insertar el mismo registro, procedemos a borrarrlo

In [19]:
with engine.begin() as connection:
    QUERY = """
    DELETE FROM PLANTILLA WHERE EMPLEADO_NO = 1234
    """
    result = connection.execute(text(QUERY))
print(f' se ha modificado {result.rowcount} fila(s)')

 se ha modificado 1 fila(s)


##### 60. Insertar la misma fila anterior sin indicar en que campos se insertan. )Por qué no se indican?

In [20]:
with engine.begin() as connection:
    QUERY = """
    INSERT INTO PLANTILLA
    VALUES (22, 2, 1234, 'Garcia J.',NULL,NULL , 3000000)
    """
    result = connection.execute(text(query))
print(f' se ha modificado {result.rowcount} fila(s)')
# es aceptable insertar valores asi

 se ha modificado 1 fila(s)


In [21]:
# cerrar la conexion
engine.dispose()

# Autor
### Oscar Gutierrez Leal
15 - 03 - 2024